# Project Part 2: Benchmark and Optimization

In this milestone you will provide a report which includes the results of your benchmark and optimization.  Your report will include a performance benchmark on a single core and then a description of what you did to speed up the code and a graph of how much improvement you made over the benchmark.  Your final report should include the following:

- Project Title
- Motivating Image
- Abstract
- Methodology
- Timing Results
- Concluding Discussion and Future Work
- References


To help you out your instructor has provided the following template


---- START TEMPLATE ----

# Part 2 - Project Title 

By "Justin Wijaya"

&#9989; Replace the following with a picture that "defines" your project.  This could be software logo, an expected outcome of your project, or a graphical representation of the research area. Again, it is okay to use the picture from the proposal but it is also good to come up with a new unique picture.  

<img alt="Simple Icon of a camera. This is just a place holder for your image" src="https://miro.medium.com/v2/resize:fit:720/1*VSQ0XEywxSgZBwW05GsZtw.png" width="20%">

Image from: [https://navaneethsdk.medium.com](https://navaneethsdk.medium.com/getting-started-with-pytorch-4fa1a5ee1502)

---
# Abstract

&#9989;  Provide a short paragraph about the software and scientific/engineering domain you cover in your project (research area, sports, economics, etc). Explain why you picked this domain (i.e. what is your motivation). Explain how computation is used in this domain. Give a short description of what you did and a summary of results.  

This project covers the domain of image classification using the Fashion-MNIST dataset. The motivation stems from its relevance in e-commerce and fashion recommendation systems, in which accurately categorizing clothing is essential. In this domain, computation is used to train the model, evaluate the results, optimize parameters, and predict using new data. ______ computation, description, summary results

---
# Methodology

&#9989;  Provide a more detailed description about what you did for this project.  Include detailed instructions needed to reprodue the project.  These instructions should be detailed enough for a future student to be able to read the report, reproduce your results and pick up where you left off. 

---
# Timing Results

&#9989;  Show the results of a baseline experiment on a single core and after optimization.  Also include a graph of the results. 


&#9989;  Provide the results of a benchmark or scaling study for your project.  Make sure you include a description of the hardware that was used and graph the results.  Make sure you include detailed descriptions about the hardware that was used.  Graphs alone are not sufficient, explain the graphs. Did they meet expectations?  Was there any anomalies?

---
# Concluding Discussion and Future Work

&#9989;  Give another short description of the project and your final results.  Use this to talk about what you learned in this project.  Include what you found interesting and what would be a next step.  

---
# References

&#9989;  Include links to websites and resources used in this project.  

---- END TEMPLATE ----

-----
### Congratulations, you are done!

Now, you just need to create a second directory in your git repository and include your report as an md or ipynb file in the directory along with any additional figures and files needed to reproduce the results.  You instructor should already have your git repository and be able to pull in your changes. 

Written by Dr. Dirk Colbry, Michigan State University (Updated by Dr. Nathan Haut in Spring 2025)
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----